<a href="https://colab.research.google.com/github/vf-fontes/intro-cd/blob/master/20200930_Olist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OLIST E-COMMERCE
Autor: Vinicius Fontes

OBJETIVO:  Realizar análise descritiva de um conjunto de Datasets do Marketplace OLIST. Aprtir desta análises verificar possibilidade de utilização de algoritmos de Machine Learning para conclusões preditivas.

*   Dataset Original: *https://www.kaggle.com/olistbr/brazilian-ecommerce*

Lista de Datasets:
*  olist_orders_dataset: Informações sobre as compras realizadas => **[DATASET PRINCIPAL]**
* olist_customers_dataset: Informações sobre os clientes bem como sua localização
* olist_geolocation_dataset: Códigos postais das cidades brasileiras e suas geolocalizações
* olist_order_items_dataset: Informações sobre produtos comprados em cada compra
* olist_order_payments_dataset: Informações sobre formas de pagamentos
* olist_order_reviews_dataset: Avaliações realizadas pelos clientes
* olist_products_dataset: Informação sobre os produtos à venda
* olist_sellers_dataset: Informações sobre o vendedor
* product_category_name_translation: Nomes dos produtos em lingua inglesa


# Instalando Bilbiotecas

In [133]:
#!pip install gitly
#!pip install fuzzywuzzy
#!pip install pyufbr
!pip install unidecode

# Importando Bibliotecas

In [134]:
# Bibliotecas de Métodos Necessários
import pandas as pd
import numpy as np
from gitly.colab.plot import GitlyPlotter
import plotly.express as px
from pyUFbr.baseuf import ufbr
from unidecode import unidecode

# Importando Datasets

In [135]:
df_clientes = pd.read_csv('/content/drive/My Drive/MBA/MÓDULO - FUNDAMENTOS/TIAGO [Introdução a Ciência de Dados]/Datasets/olist_customers_dataset.csv')
df_geo = pd.read_csv('//content/drive/My Drive/MBA/MÓDULO - FUNDAMENTOS/TIAGO [Introdução a Ciência de Dados]/Datasets/olist_geolocation_dataset.csv', encoding='utf8')
df_compra_itens = pd.read_csv('/content/drive/My Drive/MBA/MÓDULO - FUNDAMENTOS/TIAGO [Introdução a Ciência de Dados]/Datasets/olist_order_items_dataset.csv')
df_compra_pagamentos = pd.read_csv('/content/drive/My Drive/MBA/MÓDULO - FUNDAMENTOS/TIAGO [Introdução a Ciência de Dados]/Datasets/olist_order_payments_dataset.csv')
df_reviews = pd.read_csv('/content/drive/My Drive/MBA/MÓDULO - FUNDAMENTOS/TIAGO [Introdução a Ciência de Dados]/Datasets/olist_order_reviews_dataset.csv')
df_compras = pd.read_csv('/content/drive/My Drive/MBA/MÓDULO - FUNDAMENTOS/TIAGO [Introdução a Ciência de Dados]/Datasets/olist_orders_dataset.csv')
df_produtos = pd.read_csv('/content/drive/My Drive/MBA/MÓDULO - FUNDAMENTOS/TIAGO [Introdução a Ciência de Dados]/Datasets/olist_products_dataset.csv')
df_vendedores = pd.read_csv('/content/drive/My Drive/MBA/MÓDULO - FUNDAMENTOS/TIAGO [Introdução a Ciência de Dados]/Datasets/olist_sellers_dataset.csv')

# Explorando Dataset de Clientes (Customers)

## Pré-processamento

### Verificando existência de valores nulos

In [136]:
# Verificando existência de valores nulos
df_clientes.isnull().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

## Conhecendo o Dataset

In [137]:
# Conhecendo o Dataset
df_clientes.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [138]:
# Quais cidades tem no meu dataset
customerCity = df_clientes.customer_city.unique()
print('Cidades únicas encontradas no dataset: ', np.sort(customerCity))

Cidades únicas encontradas no dataset:  ['abadia dos dourados' 'abadiania' 'abaete' ... 'zacarias' 'ze doca'
 'zortea']


In [139]:
# Quantidade de cidades únicas
print('Quantidade de cidades Únicas: ',len(customerCity))

Quantidade de cidades Únicas:  4119


In [140]:
# Porcentagem de cidades brasileiras já atendidas pela OLIST
print('Porcentagem de cidades brasileiras já atendidas pela OLIST: %.2f%%'%(len(customerCity)*100/5570))

Porcentagem de cidades brasileiras já atendidas pela OLIST: 73.95%


Oportunidade de divulgação em %26,05 das cidades brasileiras, que equivale 1.450 cidades

In [150]:
# Quais cidades brasileiras não foram contempladas
cities_estate = []
for i in customerEstate:
  cities_estate.append(ufbr.list_cidades(i))
cities_estate[1][:5]

['ABDON BATISTA', 'ABELARDO LUZ', 'AGROLÂNDIA', 'AGRONÔMICA', 'ÁGUA DOCE']

In [142]:
cities = []
for i in customerEstate:
  n +=1
  for a in cities_estate[][:]:
    m +=1
    cities.append(cities_estate[n][m])
cities

SyntaxError: ignored

In [ ]:
# Quais estados tem no meu dataset
customerEstate = df_clientes.customer_state.unique()
print('Estados únicos encontradas no dataset: ', np.sort(customerEstate))

In [ ]:
# Quantidade de estados únicos
print('Quantidade de estados Únicos: ',len(customerEstate))

In [ ]:
# Quantidade de clientes
print('Quantidade de clientes que já realizaram compras pela OLIST: ', len(df_clientes.customer_unique_id.unique()))

In [ ]:
df_clientes.groupby(by='customer_city').count().sort_values(by='customer_id', ascending=False)

In [ ]:
df_temp = df_clientes.groupby( by = 'customer_unique_id' ).count()
df_temp.sort_values(by='customer_id', ascending=False)

In [ ]:
df_clientes.groupby( by = 'customer_unique_id' ).count()['customer_id'] > 2

In [ ]:
df_group = df_clientes.groupby( by = 'customer_unique_id' ).count()
df_group[ (df_group['customer_id'] >= 4) ]

In [ ]:
df_clientes.customer_city.value_counts().head(7).reset_index()

In [ ]:



gitly = GitlyPlotter(renderer='colab') #renderer='github' permite que os gráficos dinâmicos fiquem estaticos para aparecerem no notebook  do github
data = df_clientes.customer_city.value_counts().head(7).reset_index()
fig = px.bar(data,x='index', y='customer_city')
gitly.show(fig)